In [1]:
import openai
from qdrant_client import QdrantClient

## Retrieval function

In [11]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

qdrant_client = QdrantClient(url="http://localhost:6333")

def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embeddings(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
    }


In [12]:
retrieved_context = retrieve_data("What kind of shoes can I get?", qdrant_client, k=10)

## Process context format

In [14]:
def process_context(context):
    formatted_context = ""

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context

In [17]:
processed_context = process_context(retrieved_context)

## Prompt Function

In [22]:
def build_prompt(processed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{processed_context}

Question:
{question}
"""

    return prompt

In [23]:
prompt = build_prompt(processed_context, "What kind of shoes can I get?")

In [24]:
print(prompt)


You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- B0B17XG5V7: Ermonn Women's Slingback Chunky Heels Closed Toe Low Heel Pumps Woven Knit Strap Backless Mules Casual Dress Shoes Rubber sole ❤ [ HEEL HEIGHT ]: Self-covered low heel, 1.7" heel/4.5cm (approx) ❤ [ MATERIAL ]: Faux Suede/ Woven Knit Textile Material Upper; Rubber Sole. The upper is soft and not easily deformed, the inside is breathable and comfortable, suitable for spring summer and autumn ❤ [ FEATURES ]: Almond toe; chunky block heel; woven knit upper; pulls on; elastic back strap; backless shoes. Chic and comfy low heel mules, more breathable, lightweight and stable than regular heels and you can walk easily for long periods. Elastic back straps, better fixation, less likel

## Generate Answer

In [30]:
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )

    return response.choices[0].message.content

## RAG pipeline

In [31]:
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer


In [32]:
rag_pipeline("what type of shoes can I buy for kids?")

"You can buy adidas Kids' Big (8-12 Years) Freak Molded Cleats for kids. These are football cleats designed for young players, featuring a mid-cut upper for quick movement and a flexible midsole for comfort. Additionally, there is a kids' pilot costume flight suit available, but for shoes specifically, the adidas football cleats are the option listed for kids."